Current tasks
-------------
- Set _eval_energy and eval_energy to be nogil
- ~~Make CompiledModel a cdef class~~
- Link CompiledModel-based energy evaluation into the main solver
- Decide how/if to store gradients and Hessians of RedlichKisterSums
- Write analagous eval_gradient and eval_hessian functions
- Link gradient and Hessian for CompiledModel into main solver

In [1]:
from pycalphad.core.compiled_model import CompiledModel
from pycalphad import calculate, Database, Model
from pycalphad.core.phase_rec import PhaseRecord
import pycalphad.variables as v
import numpy as np

In [2]:
dbf = Database('alcocr-sandbox.tdb')

In [5]:
cmpmdl = CompiledModel(dbf, ['AL', 'CO', 'CR', 'VA'], 'BCC_A2')
out = np.zeros(7)
%time cmpmdl.eval_energy_gradient(out, np.array([101325, 600, 0.3, 0.3, 0.4-1e-12, 1e-12, 1]), np.array([]))
out

CPU times: user 283 µs, sys: 0 ns, total: 283 µs
Wall time: 253 µs


array([  0.00000000e+00,  -4.12656665e+01,  -1.38124028e+04,
        -1.57612220e+04,   4.17600426e+02,  -1.46441799e+05,
        -3.40583248e+04])

In [6]:
mdl = Model(dbf, ['AL', 'CO', 'CR', 'VA'], 'BCC_A2')
#mdl.models['idmix'] = 0
#mdl.models['ref'] = 0
#print(mdl.GM.diff(v.Y('BCC_A2', 0, 'VA')))
[mdl.GM.diff(x).subs({v.T: 600., v.P:101325., v.Y('BCC_A2', 0, 'AL'): 0.3, v.Y('BCC_A2', 0, 'CO'): 0.3,
                    v.Y('BCC_A2', 0, 'CR'): 0.4-1e-12, v.Y('BCC_A2', 0, 'VA'): 1e-12, v.Y('BCC_A2', 1, 'VA'): 1.})
 for x in [v.P, v.T, v.Y('BCC_A2', 0, 'AL'),v.Y('BCC_A2', 0, 'CO'),v.Y('BCC_A2', 0, 'CR'),v.Y('BCC_A2', 0, 'VA'), v.Y('BCC_A2', 1, 'VA')]]

[0,
 -41.2656665260401,
 -13812.4027714589,
 -15761.2219707636,
 417.600425887349,
 -146441.799331453,
 -34058.3248161477]

In [ ]:
%time res = calculate(dbf, ['AL', 'CO', 'CR', 'VA'], sorted(dbf.phases.keys()), T=2000, P=101325, output='GM', pdens=200, model=CompiledModel)
res.GM[..., :20]

In [ ]:
%time res2 = calculate(dbf, ['AL', 'CO', 'CR', 'VA'], sorted(dbf.phases.keys()), T=2000, P=101325, output='GM', pdens=200, model=Model)
res2.GM[..., :20]